# Внутренние испытания 18 июня 2020
* Новый БП 5В + 12В (с нагрузкой на связке резисторов)
* Новая прошивка платы
* Исправлены контакты (все еще на пружинках), все каналы работают
* Реализована подстройка офсетов и вольтажей

In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re
import os
import requests

In [4]:
URL = 'http://192.168.88.155:8081/logs'
def show_files():
    return(requests.get(URL).json())
def download(filename):
    r = requests.get(URL + '/' + filename)
    with open(filename, 'wb') as f:
        f.write(r.content)

In [5]:
def generate_dataframes_and_comments_from_file(filename):
    dt = {'time' : [], 'ph': []}
    dt_comments = {'time' : [], 'comment': []}
    with open(filename) as fp:
        line = fp.readline()
        last_value = 0
        while line:
            m = re.search(r'\[(\d{2}\:\d{2}\:\d{2})\]\:\s(\d+)$', line)
            if m:
                last_value = m.group(2)
                dt['time'].append(m.group(1))
                dt['ph'].append(last_value)
            else:
                m = re.search(r'\[(\d{2}\:\d{2}\:\d{2})\]\:\s(.+)$', line)
                if m:
                    time = m.group(1)
                    text = m.group(2)
                    if time in dt_comments['time']:
                        pass
                        """
                        for i in range(len(dt_comments['time'])):
                            if dt_comments['time'][i] == time:
                                dt_comments['comment'][i] += ", " + text
                        """
                    else:
                        dt_comments['time'].append(time)
                        dt_comments['comment'].append(text)
                    if time not in dt['time']:
                        dt['time'].append(time)
                        dt['ph'].append(last_value)
            line = fp.readline()
    df = pd.DataFrame(data = dt)
    return(df, dt_comments)

In [6]:
def draw_file(filename):
    df, comments = generate_dataframes_and_comments_from_file(filename)
    fig = go.Figure([go.Scatter(x=df['time'], y=df['ph'])])
    for i in range(len(comments['time'])):    
        fig.add_annotation(go.layout.Annotation(
            x = comments['time'][i],
            text = comments['comment'][i],
            align='center',
            showarrow=False,
            yanchor='bottom',
            textangle=-90))
    fig.show()

14:02 Откалибровано. Первый запуск на растворах NaOH и кислоты.

14:08 Увеличил ADC range до 3В

14:10 Уменьшил ADC range до 1В

14:10 Уменьшил ADC range до 0.5В (наблюдается слишком сильный размах - засветка)

14:14 Увеличил ADC до 0.75В (наблюдается слишком темное изображение и слабая амплитуда)

14:16 обнаружили что земли не соединены. Соединили. Все стало засвечено белым 

14:18 увеличил ADC до 2В - слишком малая амплитуда

14:20 уменьшил ADC до 1.6 - слишком высокая амплитуда

14:22 увеличил до 1.8 - слишком малая амплитуда

14:24 уменьшил до 1.7

14:27 уменьшил до 1.65 - самая отчетливая картина, однако большее влияние по показатели оказывает муар, проявляющийся при подаче OH-

14:29 vb1=vb2=2.00 В (изначально было 1.65) - изменений нет

14:32 vb1=vb2=3.00 В - картина изменений стала менее выраженной

14:35 vb1=vb2=1В - картина изменений стала более выраженной

14:37 vb1=vb2=0.5В - еще более ярко выраженный размах

14:39 vb1=vb2=0.25В - разницы не заметно

14:43 vb1=0.25, vb2=0.5

14:46 vb1=0.25, vb2=1

Вывод - достигнута амплитуда (13.1k при OH-, 2.3k при H+). Но при OH- очень высокое влияние на значения оказывает муар, а не PH среды. При постоянном PH щелочи имеется лесенка близкая к гиперболе, что не соответствует изменению PH показателя раствора.



In [61]:
download('sequence.py-2020.06.18_14:03:02.log')
draw_file('sequence.py-2020.06.18_14:03:02.log')

15:01 новая циклограмма на тех же растворах adc=1.65, vb=0.25, vb2=0.25
Очень сильно проявляется муар

In [62]:
download('sequence.py-2020.06.18_15:00:16.log')
draw_file('sequence.py-2020.06.18_15:00:16.log')

15:25 долил в W2 NaHCO3 (избыток) и NaSO4. Должен получиться более насыщенный ионами щелочной раствор. Выполнена продувка для перемешивания растворов. Чип 1 вышел из строя, в углу, по которому калибровка - шум. Поменяли чип на 2 - странно реагирует третий канал, нелинейно. И изображение слишком белое. Чип испорчен. Вернули обратно чип 1.

Чип: 1
adc_range = 1.65 В
vb1=vb2=1.4 В, на более низких не калибруется.
Калибровка done

17:00 начал поднимать ADC range и поднял до 2. Стало хуже.

17:04 начал опускать ADC range

17:06 опустил до 1.8, в левом верхнем углу стали проявляться ярко-белые полоски. Оставил на 1.9

17:07 ADC=1.8

17:09 ADC=1.7 - проявился шум в виде белых точек в нижней половине изображения

17:10 ADC=1.65

17:13 поднял напряжение до 1.65

Во всех случаях муар продолжает проявляться (в верхней левой части, полосками). В нижней части какой-то рандом.

Заключение: муар не зависит от насыщения PH-раствора ионами.

In [77]:
download('sequence.py-2020.06.18_16:49:02.log')
draw_file('sequence.py-2020.06.18_16:49:02.log')